In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
from IPython.display import display, Math

#alteração 2

In [ ]:
def DeletarObj(nome, lum):
    lum.select(nome)
    lum.delete()

In [ ]:
def CriarRect(nome, Xmin, Xmax, Y, Yspan, Zmin, Zmax, material, lum):
    
    lum.addrect()
    lum.set({
        "name": nome,
        "x min": Xmin,"y": Y,"z min": Zmin,
        "x max": Xmax, "y span": Yspan, "z max": Zmax,
        "material": material
        })

In [ ]:
def CriarTP(nome, Xmin, Xmax, Y, Yspan1, Yspan2, Zmin, Zmax, material, lum):

    X = (Xmin + Xmax)/2
    Xspan = Xmax-Xmin
    Z = (Zmin + Zmax)/2
    Zspan = Zmax-Zmin

    lum.addobject("linear_taper")
    lum.set({
        "name": nome,
        "x": X, "y": Y, "z": Z,
        "thickness": Zspan, 
        "angle_side": 90, 
        "width_l": Yspan1,
        "width_r": Yspan2,
        "len": Xspan,
        "material": material
    })

In [ ]:
def CriarMesh(nome, Xmin, Xmax, Ymin, Ymax, Zmin, Zmax, propX, propY, propZ, lum):
    lum.addmesh()
    lum.set({
        "name": nome,
        "x min": Xmin,"y": Y,"z min": Zmin,
        "x max": Xmax, "y span": Yspan, "z max": Zmax,
        "set equivalent index": True,
        "equivalent x index": propX, "equivalent y index": propY, "equivalent z index": propZ
    })


In [ ]:
def altRect(nome, Xmin, Xmax, Y, Yspan, Zmin, Zmax, material, lum):
    DeletarObj(nome, lum)
    CriarRect(nome, Xmin, Xmax, Y, Yspan, Zmin, Zmax, material, lum)

In [ ]:
def altTP(nome, Xmin, Xmax, Y, Yspan1, Yspan2, Zmin, Zmax, material, lum):
    DeletarObj(nome, lum)
    CriarTP(nome, Xmin, Xmax, Y, Yspan1, Yspan2, Zmin, Zmax, material, lum)

In [ ]:
def altMesh(nome, Xmin, Xmax, Ymin, Ymax, Zmin, Zmax, propX, propY, propZ, lum):
    DeletarObj(nome, lum)
    CriarMesh(nome, Xmin, Xmax, Ymin, Ymax, Zmin, Zmax, propX, propY, propZ, lum)

In [ ]:

inicioMMI = 0
compMMI = 37.2947e-6
largMMI = 6e-6
zmin = 0
zspan = 220e-9
ycentro = 0


largWG = .45e-6
bufferWG = 4e-6
compTP = 5e-6
largTP = 0.609367e-6
posSaida = 1.5e-6

material = "Si (Silicon) - Palik";
BGmaterial = "SiO2 (Glass) - Palik";


inicTPS = compMMI + inicioMMI
fimTPE = compTP + inicTPS
fimTPS1 = compTP + inicTPS
fimTPS2 = compTP + inicTPS
fimTPS3 = compTP + inicTPS
largTPE = largTP
largTPS1 = largTP
largTPS2 = largTP
largTPS3 = largTP
posSaida1 = ycentro + posSaida + 0.59e-6
posSaida2 = ycentro
posSaida3 = ycentro - posSaida - 0.55e-6



In [ ]:

nmodosTP = 10
nmodosMMI = 100
mcy = 25
mcz = 10

In [ ]:
lum.switchtolayout()

altRect(nomes[0], inicioMMI, compMMI, ycentro, largMMI, zmin, zspan, material, lum)
altRect(nomes[1], inicioMMI-(compTP+bufferWG), inicioMMI, ycentro, largWG, zmin, zspan, material, lum)
altRect(nomes[2], inicTPS, fimTPS1, posSaida1, largWG, zmin, zspan, material, lum)
altRect(nomes[3], inicTPS, fimTPS2, posSaida2, largWG, zmin, zspan, material, lum)
altRect(nomes[4], inicTPS, fimTPS3, posSaida3, largWG, zmin, zspan, material, lum)
altTP(nomes[5], inicioMMI-compTP, inicioMMI, ycentro, largWG, largTPE, zmin, zspan, material, lum)
altTP(nomes[6], inicTPS, fimTPS1, posSaida1, largTPS1, largWG, zmin, zspan, material, lum)
altTP(nomes[7], inicTPS, fimTPS2, posSaida2, largTPS2, largWG, zmin, zspan, material, lum)
altTP(nomes[8], inicTPS, fimTPS3, posSaida3, largTPS3, largWG, zmin, zspan, material, lum)



In [ ]:

nomes = [["MMImeio", "Entrada", "Saida1", "Saida2", "Saida3", "EntradaTP", "Saida1TP", "Saida2TP", "Saida3TP"], ["mesh"]]
X = [[][]]

In [ ]:
DeletarObj("EME", lum)

lum.addeme()
lum.set({
    "allow custom eigensolver settings":1, 
    "display cells":1, 
    "number of cell groups":3,

    "y min bc": "PML",
    "y max bc": "PML",
    "z min bc": "Symmetric",
    "z max bc": "PML"
})

lum.set({
    "mesh cells y": mcy,
    "mesh cells z": mcz,
    "group spans": np.transpose([compTP, compMMI, compTP]),
    "cells":np.transpose([ncellsTP, 1, ncellsTP]),
    "subcell method":np.transpose([1, 0, 1]), # 0 = none,  1 = CVCS
    "modes":np.transpose([nmodosTP, nmodosMMI, nmodosTP]),
})

lum.set({
    "x min": inicioMMI-(compTP), "y":ycentro, "z": zmin+(zspan/2),
    "y span": largMMI*1.25, "z span": zspan,
})



lum.set("background material", BGmaterial)
lum.select("EME::Ports::port_1")
lum.set({
    "port location": "right",
    "use full simulation span": 0,
    "y": posSaida1, "z": zmin +(zspan/2),
    "y span": 5e-6, "z span": 3e-6
})
lum.select("EME::Ports::port_2")
lum.set({
    "port location": "right",
    "use full simulation span": 0,
    "y": posSaida2, "z": zmin +(zspan/2),
    "y span": 5e-6, "z span": 3e-6
})
lum.select("EME::Ports::port_3")
lum.set({
    "port location": "right",
    "use full simulation span": 0,
    "y": posSaida3, "z": zmin +(zspan/2),
    "y span": 5e-6, "z span": 3e-6
})


In [ ]:
# Body config------------------------------------
BdX_Span = 31.87e-6 # Body X Span
BdY_Span = 6e-6 # Body y Span


# Ports config-----------------------------------

OutMx = [1.6e-6,-1.6e-6] # Array of output ports y positions (put 0 on first element to ignore this line)
InMx = [0]               # Array of input ports y positions (put 0 on first element to ignore this line) 

# If the lines above has to be ignored, use these ones instead: 
InP = 1 # Number of Input ports
OutP = 2 # Number of Output ports
Asy = 0 # Force asymmetry on inputs ports auto-displacement (0 -> Symetry, 1 -> Asymmetry)
         
PYSpan_in = 12e-6 # Input Port Y Span 
PYSpan_out = 3e-6 # Output Port Y Span 


# Tappers config---------------------------------
SetToTappers = 1 # Set tappers as inputs/outputs
TLen = 4e-6 # Tapper Length
TWidth = 1.1e-6 # Tapper Width


# Mesh config------------------------------------
MeshEnableIn = 0 # enable input mesh insertion
MeshEnableOut = 1 # enable output mesh insertion
MeshType = 1 # 0 for Override, 1 for Multipler

MeshOver = [[0.05e-6,0.05e-6,0.05e-6] , [0.05e-6,0.05e-6,0.05e-6]] # Override mesh value dx,dy and dz ([[Input Mesh],[Output Mesh]])
MeshMultiplier = [[2,2,2] , [4,4,4]] # Mesh multipler value dx,dy and dz ([[Input Mesh],[Output Mesh]])


# EME config-------------------------------------
EME_MC = 100 # Number of EME Mesh Cells (Y and Z)
SideCells = 4 # Number of Cells on Input/Output groups